In [ ]:
import cv2
import numpy as np
import os

### MATRIZ BCH

Esta matríz se ha obtenido del artículo científico: `Brightness Calculation in Digital Image Processing` y nos ayudará a calcular el brillo de una imagen.

In [ ]:
MATRIZ_toDEF = np.array([[0.2053, 0.7125, 0.4670],
                        [1.8537, -1.2797, -0.4429],
                        [-0.3655, 1.0120, -0.6014]])

print(MATRIZ_toDEF) 

### 🧮 Cálculo del Brillo utilizando la Métrica de Cohen  

La función `obtener_brillo` calcula el brillo de una imagen a partir de sus componentes en el espacio de color XYZ, utilizando la métrica de Cohen basada en el modelo BCH (Brightness, Chroma, Hue).  

**¿Cómo funciona?**  
- Transformación BCH:  
   - Se toma la imagen en sus componentes **X, Y, Z** (R, G, B) y se aplica la matriz auxiliar para obtener nuevas representaciones **D, E, F**.    

- Cálculo brillo:
   - A partir de los valores **D, E, F**, se calcula el brillo **B** como la norma euclidiana en este espacio:  
     
     ```math
     B = \sqrt{D^2 + E^2 + F^2}
     ```
   - Luego, se obtiene el **brillo promedio**  de toda la imagen.  


In [ ]:
def calc_metricas_cohen(X, Y, Z):
    alto, ancho = X.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
    # Aplicar la matriz DEF
    cohen = pixels_xyz @ MATRIZ_toDEF.T  # Resultado (N, 3)
    return cohen.reshape(alto, ancho, 3)

def obtener_brillo_valores_def(X, Y, Z):
    cohen = calc_metricas_cohen(X, Y, Z)
    D, E, F = cohen[:, :, 0], cohen[:, :, 1], cohen[:, :, 2]
    B = np.sqrt(D**2 + E**2 + F**2)
    return B.mean(), D, E, F

### 🧮 Obtener brillo y canal DEF de la imagen

La función `obtener_brillo_imagen` aplica las funciones creadas en la celda anterior y devuelve el valor del brillo.

La función `obtener_canal_def` aplica las funciones creadas en la celda anterior y devuelve el valor del canal DEF.

**¿Cómo funciona?**  
- Separa la imagen en RGB
- Le pasa cada componente a la función obtener brillo
- Devuelve el brillo de la imagen y el canal DEF en las respectivas funciones

In [ ]:
def obtener_brillo_imagen(imagen_rgb):
    imagen_float = imagen_rgb.astype(np.float32) / 255.0

    X, Y, Z = cv2.split(cv2.cvtColor(imagen_float, cv2.COLOR_RGB2XYZ))
    
    brillo = obtener_brillo_valores_def(X, Y, Z) [0]
    return brillo

### 🧮 Cargar imagen RGB   

La función `cargar_imagen_rgb` es una función auxiliar que carga una imagen en rgb .  


In [ ]:
def cargar_imagen_rgb(ruta):
    img = cv2.imread(ruta)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

### 🗂️ Procesamiento masivo de imágenes calculo del brillo

Esta función `procesar_imagenes_carpeta_bch` aplica la métrica de Cohen para calcular el brillo a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Carga la imagen en RGB.
- Aplica la métrica de cohen para obtenter el brillo .
- Guarda el valor del brillo obtenido en un archivo `.txt`.

In [ ]:
def procesar_imagenes_carpeta_bch(
        carpeta_entrada="images",
        carpeta_salida="images_procesadas"
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)

        # Añadimos la subcarpeta específica para 'bch'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "bch")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar los brillos
        ruta_txt = os.path.join(carpeta_salida_sub, "info_brillos_bch.txt")

        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Nivel de Brillo (método BCH)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Valores mayores corresponden a mayor brillo.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(".png"):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en rgb
                    img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                    
                    # Calcular brillo
                    brillo = obtener_brillo_imagen(img_rgb)
                    
                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> brillo: {brillo:.4f}\n")

    print("Procesamiento completado con método BCH.")

In [ ]:
procesar_imagenes_carpeta_bch(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
)

### 🧮 Modificar brillo de una imagen   

La función `modificar_brillo` permite ajustar el brillo de una imagen.  

**¿Cómo funciona?**  
- Se determina un coeficiente al llamar a la función 
- Le pasa cada componente a la función obtener brillo
- Devuelve el brillo de la imagen

In [ ]:
# m= valores en el scrollbar entre 0.5 y 1.5 (modificar si vemos que aumenta/disminuye demasiado)

def modificar_brillo(rgb, m0=1):
    #algoritmo tv_based
    rgb_nuevo = np.array([rgb[0] * m0, rgb[1] * m0, rgb[2] * m0])
    return rgb_nuevo